# Imports

In [ ]:
import geopandas as gpd
import pandas as pd
from pathlib import Path

# Functions

In [ ]:
def add_sum_col(df):
    cols = df.columns
    for idx, col in enumerate(cols):
        df.loc['Summe',col] = df.iloc[:,idx].sum()

# Parameters

In [ ]:
sbw_path = Path(r"H:\2 Projekte\8000-\8500er\8532\8532.10 GEP Richterswil"
    r"\05 Berechnungen Grundlagen\8 Entwässerungskonzept\SBW im EZG\240124 Richterswil SK\Alle_exp_SBW\Exports\Teileinzugsgebiet-SBW_Ist.shp")
quartiere_path = Path(r"H:\2 Projekte\8000-\8500er\8532\8532.10 GEP Richterswil"
    r"\05 Berechnungen Grundlagen\8 Entwässerungskonzept\Bevölkerungsanalyse\Quartieranalyse\Daten\Quartieranalyse\QUARTIERE_F.shp")
sbw_tabelle = Path(r"H:\2 Projekte\8000-\8500er\8532\8532.10 GEP Richterswil"
    r"\05 Berechnungen Grundlagen\8 Entwässerungskonzept\SBW im EZG\Richterswil_SBW-Tabelle.csv")

save_dir = Path(r"H:\2 Projekte\8000-\8500er\8532\8532.10 GEP Richterswil\05 Berechnungen Grundlagen\8 Entwässerungskonzept\SBW im EZG")

# Abflussbeiwert Analysis

In [ ]:
ezg_path = Path(r"H:\2 Projekte\8000-\8500er\8532\8532.10 GEP Richterswil\05 Berechnungen Grundlagen"
    r"\8 Entwässerungskonzept\SBW im EZG\240124 Richterswil SK\Alle_exp_SBW\Exports\EZG_ist.csv")
ezg = pd.read_csv(ezg_path, sep=';')

# Compute AB / BG gesamt
ezg['abflussbeiwert_gesamt_ist'] = ezg['abflussbeiwert_sw_ist'] + ezg['abflussbeiwert_rw_ist']
ezg['befestigungsgrad_gesamt_ist'] = ezg['befestigungsgrad_sw_ist'] + ezg['befestigungsgrad_rw_ist']

# Rename WG2
mask = (ezg['abflussbeiwert_gesamt_ist'] == 35) & (ezg['bezeichnung'] == 'Zweigeschossige Wohn')
ezg.loc[mask, 'bezeichnung'] = 'Zweigeschossige Wohnzone mit Gewerbeanteil'

# Rename W2
mask = ezg['bezeichnung'] == 'Zweigeschossige Wohn'
ezg.loc[mask, 'bezeichnung'] = 'Zweigeschossige Wohnzone'

# Rename WG3
mask = (ezg['abflussbeiwert_gesamt_ist'] == 65) & (ezg['bezeichnung'] == 'Dreigeschossige Wohn')
ezg.loc[mask, 'bezeichnung'] = 'Dreigeschossige Wohnzone mit Gewerbeanteil'

# Rename W3
mask = ezg['bezeichnung'] == 'Dreigeschossige Wohn'
ezg.loc[mask, 'bezeichnung'] = 'Dreigeschossige Wohnzone'

In [ ]:
ab_bg_ezg = ezg[['bezeichnung', 'abflussbeiwert_gesamt_ist', 'befestigungsgrad_gesamt_ist']].copy(deep=True)

ab_bg_ezg = ab_bg_ezg.groupby('bezeichnung').mean()
ab_bg_ezg

In [ ]:
save_path = save_dir / 'EZG-Ist_Vergleich-AB-BG.xlsx'
ab_bg_ezg.to_excel(save_path)

# Einwohnerdichte Analysis

In [ ]:
name = 'bezeichnung'
ew_ist = 18000
ew_planung = 28000

ewd_ezg = ezg.groupby(name)[['einwohnerdichte_ist']].mean()
ewd_ezg.rename(columns={'einwohnerdichte_ist': 'einwohnerdichte_gep-2005'}, inplace=True)
ewd_ezg['flaeche'] = ezg.groupby(name)[['flaeche']].sum()
ewd_ezg['einwohner_gep-2005'] = ewd_ezg['einwohnerdichte_gep-2005'] * ewd_ezg['flaeche']
ewd_ezg['einwohner_ist-2024'] = ewd_ezg['einwohner_gep-2005'] * ew_ist / ewd_ezg['einwohner_gep-2005'].sum()
ewd_ezg['einwohnerdichte_ist-2024'] = ewd_ezg['einwohner_ist-2024'] / ewd_ezg['flaeche']
ewd_ezg['einwohner_planung-2024'] = ewd_ezg['einwohner_gep-2005'] * ew_planung / ewd_ezg['einwohner_gep-2005'].sum()
ewd_ezg['einwohnerdichte_planung-2024'] = ewd_ezg['einwohner_planung-2024'] / ewd_ezg['flaeche']
ewd_ezg.loc['SUMME', 'einwohner_gep-2005'] = ewd_ezg['einwohner_gep-2005'].sum()
ewd_ezg.loc['SUMME', 'einwohner_ist-2024'] = ewd_ezg['einwohner_ist-2024'].sum()
ewd_ezg.loc['SUMME', 'einwohner_planung-2024'] = ewd_ezg['einwohner_planung-2024'].sum()
ewd_ezg

In [ ]:
save_path = save_dir / 'EW_Vergleich-GEP-Ist-Planung.xlsx'
ewd_ezg.to_excel(save_path)

# Einwohner Analysis

## Read Files

In [ ]:
sbw = gpd.read_file(sbw_path)
quartiere = gpd.read_file(quartiere_path)
df_sbw = pd.read_csv(sbw_tabelle, sep=';')

In [ ]:
# sbw.rename(columns={'sbw_ref': 'SK ID'}, inplace=True)
# sbw = pd.merge(sbw, df_sbw, on='SK ID')

In [ ]:
name = 'U_ZONE_KT'
ew_bz = sbw_quartiere.groupby(name)[['einwohner_gep', 'einwohner_quartiere']].sum()
add_sum_col(ew_bz)
ew_bz

In [ ]:
# EW dichte GEP
sbw['ew_dichte_gep'] = sbw['einwohner'] / sbw.area
print(sbw['einwohner'].sum())

# EW dichte Quartieranalyse
quartiere['ew_dichte_quartiere'] = (quartiere['U_EINW'] + quartiere['U_BESCH']) / quartiere.area
print(quartiere['U_EINW'].sum() + quartiere['U_BESCH'].sum())

In [ ]:
print(sbw.geometry.type.unique())
print(quartiere.geometry.type.unique())

sbw_quartiere = sbw.overlay(quartiere, how='intersection', keep_geom_type=None)
print(sbw_quartiere.geometry.type.unique())

In [ ]:
sbw_quartiere['einwohner_gep'] = sbw_quartiere['ew_dichte_gep'] * sbw_quartiere.area
print(sbw_quartiere['einwohner_gep'].sum())

sbw_quartiere['einwohner_quartiere'] = sbw_quartiere['ew_dichte_quartiere'] * sbw_quartiere.area
print(sbw_quartiere['einwohner_quartiere'].sum())

## EW pro Sonderbauwerk

In [ ]:
sbw_quartiere.set_index('Name', inplace=True)
ew_sbw = sbw_quartiere.groupby('Name')[['einwohner_gep', 'einwohner_quartiere']].sum()
add_sum_col(ew_sbw)
ew_sbw

## EW pro Bauzone